# [Lab Session 2](otsu.ipynb)

In [10]:
import numpy as np
import matplotlib.pyplot as plt

**[Task 2.1.]()** Implement Otsu thresholding as a re-usable function:

In [14]:
def otsu_thresholding(img):
    img = (img * 255).round().astype(np.uint8)

    h = np.zeros(256)

    for i in range(256): 
        h[i] = np.sum(img == i)

    # sum_all -> gewichtete Summe der Intensitätswerte (np.dot)
    sum_all = np.dot(np.arange(256), h)
    sum_B = 0 
    w_B = 0 
    w_F = 0 
    max_variance = 0 
    optimal_threshold = 0

    # gesamtzahl der Pixel
    total_pixels = img.size

    #Iterieren über alle Werte im Array; w_B -> Gewicht des Hintergrundes, w_F -> Gewicht des Vordergrundes
    for T in range(1, 256):
        w_B += h[T - 1]
        w_F = total_pixels - w_B
        if w_B == 0 or w_F == 0:
            continue

    #Berechnung von Summe und Mittelwerten für Berechnung der Varianz 
        sum_B += (T - 1) * h[T - 1]
        mean_B = sum_B / w_B
        mean_F = (sum_all - sum_B) / w_F

        # Varianzen berechnen, Vorlesung slide 22
        var_B = np.sum(((np.arange(T) - mean_B) ** 2) * h[:T]) / w_B
        var_F = np.sum(((np.arange(T, 256) - mean_F) ** 2) * h[T:]) / w_F

    #Formel aus Forlesung, muss minimiert werden slide 21
        within_class_variance = w_B * var_B + w_F * var_F

    # Wenn within_class_variance kleiner als Varianz zuvor, dann wird treshold neu gesetzt, sowie max_variance an neuen Wert angepasst, sodass sich within_class_variance über steigende iterationen verringert 
        if within_class_variance < max_variance or max_variance == 0:
            max_variance = within_class_variance
            optimal_threshold = T
        
    return optimal_threshold


In [15]:
otsu_thresholding(img_ref)

255

**[Task 2.2.]()** Implement a re-usable function to compute the Dice coefficient:

In [17]:
img_ref = plt.imread("data/NIH3T3/gt/0.png")


def compute_dice(bin1, bin2):
    bin1 = bin1.astype(bool) 
    bin2 = bin2.astype(bool)

    dice = 2 * np.logical_and(bin1, bin2).sum()  / (bin1.sum()  + bin2.sum())

    return dice

**[Task 2.3.]()** Test your implementations from above:

In [18]:
compute_dice(img_ref, img_ref)

1.0

The result should be the same as in your [otsu.ipynb](otsu.ipynb) notebook.

**[Task 2.4.]()** Perform batch processing:

In [28]:
for i in [28, 29, 33, 44, 46, 49]: 

    # Load the i-th image 
    image = plt.imread(f'data/NIH3T3/im/dna-{i}.png') 

    # Load the corresponding ground truth 
    ground_truth = plt.imread(f'data/NIH3T3/gt/{i}.png')

    ot = otsu_thresholding(image)
    di = compute_dice(image, ground_truth)
    print(i, ot, di)

28 35 0.3158152367118249
29 41 0.28370685279662594
33 71 0.2912938620315046
44 67 0.4090528400233743
46 75 0.31283490311025464
49 46 0.4014820708339262
